In [1]:
# !pip install transformers
# !pip install datasets==2.21.0
# !pip install wandb


In [5]:
import torch
from transformers import GPT2ForQuestionAnswering, GPT2TokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
from torch.nn.utils.rnn import pad_sequence
import os
import wandb
from transformers.integrations import WandbCallback
import numpy as np

from datasets import load_metric # used in compute_metrics
from transformers.trainer_utils import EvalPrediction
from typing import Dict, List


def freeze_layers(model, variant_type):
    if variant_type == "noNorm":
        for name, param in model.named_parameters():
            if "ln" in name:
                param.requires_grad = False
    elif variant_type == "AttnOnly":
        for name, param in model.named_parameters():
            if "ln_2" in name:  # Freeze FFN layer norm
                param.requires_grad = False
    elif variant_type == "FFOnly":
        for name, param in model.named_parameters():
            if "ln_1" in name:  # Freeze attention layer norm
                param.requires_grad = False
    # For baseModel, we don't freeze any layers

def prepare_squad_dataset(tokenizer):
    dataset = load_dataset("squad")

    def preprocess_function(examples):
        questions = [q.strip() for q in examples["question"]]
        contexts = [c.strip() for c in examples["context"]]

        # Tokenize questions and contexts together
        tokenized_examples = tokenizer(
            questions,
            contexts,
            max_length=384,
            truncation="only_second",
            stride=128,
            return_overflowing_tokens=True,
            padding="max_length",
            return_offsets_mapping=True,
        )

        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping", None)
        if sample_mapping is None:
            sample_mapping = list(range(len(questions)))

        # The offset mappings will give us a map from token to character position in the original context. This will
        # help us compute the start_positions and end_positions.
        offset_mapping = tokenized_examples["offset_mapping"]

        # Let's label those examples!
        tokenized_examples["start_positions"] = []
        tokenized_examples["end_positions"] = []

        for i, offsets in enumerate(offset_mapping):
            input_ids = tokenized_examples["input_ids"][i]

            # For GPT-2, we'll use the first token as our "impossible answer" token
            impossible_answer_index = 0

            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i) if hasattr(tokenized_examples, "sequence_ids") else None

            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            answers = examples["answers"][sample_index]

            # If no answers are given, set the impossible_answer_index as answer.
            if len(answers["answer_start"]) == 0:
                tokenized_examples["start_positions"].append(impossible_answer_index)
                tokenized_examples["end_positions"].append(impossible_answer_index)
            else:
                # Start/end character index of the answer in the text.
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])

                # Start token index of the current span in the text.
                token_start_index = 0
                while sequence_ids is not None and token_start_index < len(sequence_ids) and sequence_ids[token_start_index] != 1:
                    token_start_index += 1

                # End token index of the current span in the text.
                token_end_index = len(input_ids) - 1
                while sequence_ids is not None and token_end_index >= 0 and sequence_ids[token_end_index] != 1:
                    token_end_index -= 1

                # Detect if the answer is out of the span (in which case this feature is labeled with the impossible_answer_index).
                if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                    tokenized_examples["start_positions"].append(impossible_answer_index)
                    tokenized_examples["end_positions"].append(impossible_answer_index)
                else:
                    # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                    # Note: we could go after the last offset if the answer is the last word (edge case).
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    tokenized_examples["start_positions"].append(token_start_index - 1)
                    while token_end_index >= 0 and offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    tokenized_examples["end_positions"].append(token_end_index + 1)

        return tokenized_examples

    tokenized_datasets = dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=dataset["train"].column_names,
    )

    return tokenized_datasets


from datasets import load_metric
from transformers import EvalPrediction
import numpy as np

def compute_metrics(eval_pred: EvalPrediction):
    metric = load_metric("squad")

    logits, labels = eval_pred
    start_logits, end_logits = logits

    # Convert start and end logits to predictions
    start_predictions = np.argmax(start_logits, axis=-1)
    end_predictions = np.argmax(end_logits, axis=-1)

    predictions = []
    references = []

    # Check if labels is empty or None
    if labels is None or len(labels) == 0:
        print("Warning: Labels are empty or None. Using dummy labels.")
        labels = [{"id": str(i), "answers": {"text": [""], "answer_start": [0]}} for i in range(len(start_predictions))]

    for i, (start_pred, end_pred) in enumerate(zip(start_predictions, end_predictions)):
        # Get the predicted answer
        pred_tokens = tokenizer.convert_ids_to_tokens(
            range(start_pred, end_pred + 1)
        )
        pred_text = tokenizer.convert_tokens_to_string(pred_tokens)

        # Get the actual answer
        if isinstance(labels, (list, np.ndarray)) and i < len(labels):
            example_labels = labels[i]
            if isinstance(example_labels, dict):
                true_text = example_labels.get('answers', {}).get('text', [''])[0]
                true_start = example_labels.get('answers', {}).get('answer_start', [0])[0]
            else:
                true_text = ""
                true_start = 0
        else:
            true_text = ""
            true_start = 0

        predictions.append({
            "id": str(i),
            "prediction_text": pred_text
        })

        references.append({
            "id": str(i),
            "answers": {
                "text": [true_text],
                "answer_start": [true_start]
            }
        })

    results = metric.compute(predictions=predictions, references=references)

    return {
        "f1": results["f1"],
        "exact_match": results["exact_match"],
    }


def fine_tune_model(model, tokenizer, dataset, output_dir, variant, num_train_epochs=3):
    # Initialize wandb run
    wandb.init(project=f"GPT-Valkyrie_LN-124m__{variant}__SQuAD", reinit=True)
    run_name = wandb.run.name

    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=300,
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=10,
        evaluation_strategy="steps",
        eval_steps=100,
        save_steps=200,
        load_best_model_at_end=True,
        report_to="wandb",
        run_name=run_name,
    )

    from transformers import DefaultDataCollator

    # Use default_data_collator instead of a custom one
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        tokenizer=tokenizer,
        data_collator=DefaultDataCollator(return_tensors="pt"),
        compute_metrics=compute_metrics,
        callbacks=[WandbCallback()],
    )

    trainer.train()
    wandb.finish()
    return trainer.model, run_name

In [6]:
# MAIN LOOP
wandb.login()

variants = ["noNorm", "AttnOnly", "FFNonly", "baseModel"]
base_model_path = "shng2025/GPT-Valkyrie_LN-124m__baseModel__"  # Changed to LN model

from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

dataset = prepare_squad_dataset(tokenizer)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
for variant in variants:
    print(f"Processing {variant} model...")

    # Use the correct base model for each variant
    model_path = f"shng2025/GPT-Valkyrie_LN-124m__{variant}__"
    model = GPT2ForQuestionAnswering.from_pretrained(model_path)

    freeze_layers(model, variant)

    output_dir = f"./results/{variant}"
    fine_tuned_model, run_name = fine_tune_model(model, tokenizer, dataset, output_dir, variant)

    # Save the model locally
    local_save_dir = f"./local_models/GPT-Valkyrie_LN-124m__{variant}__SQuAD"
    fine_tuned_model.save_pretrained(local_save_dir)
    tokenizer.save_pretrained(local_save_dir)
    print(f"Model saved locally to {local_save_dir}")

    # Push the model to your HuggingFace Hub repository
    new_repo_name = f"shng2025/GPT-Valkyrie_LN-124m__{variant}__SQuAD"
    fine_tuned_model.push_to_hub(new_repo_name, branch=run_name)
    tokenizer.push_to_hub(new_repo_name, branch=run_name)
    print(f"Model pushed to HuggingFace Hub: {new_repo_name}, branch: {run_name}")

Processing noNorm model...


Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at shng2025/GPT-Valkyrie_LN-124m__noNorm__ and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


You are adding a <class 'transformers.integrations.integration_utils.WandbCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
WandbCallback


Step,Training Loss,Validation Loss,F1,Exact Match
100,5.021700,4.628865,0.000000,43.155259
200,4.711100,4.411581,0.000000,53.153404
300,4.330000,4.343086,0.000000,31.163050


<ipython-input-5-cc8b5ee2a710>:125: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad")


{'exact_match': 43.15525876460768, 'f1': 0.0}
{'exact_match': 53.15340382118345, 'f1': 0.0}
{'exact_match': 31.163049526989425, 'f1': 0.0}
